<a href="https://colab.research.google.com/github/satvick2003/PYTHON-PROGRAM-TASK-3/blob/main/PYTHONWEEK3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install requests tqdm beautifulsoup4


In [ ]:
import os
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin

def download_image(url, save_dir):
    try:
        # Check if the URL is valid
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Check for HTTP errors

        # Get the image file name
        file_name = os.path.basename(urlparse(url).path)
        if not file_name:
            file_name = 'image.jpg'  # Default file name

        # Create the save directory if it doesn't exist
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        # Download the image
        file_path = os.path.join(save_dir, file_name)
        with open(file_path, 'wb') as file:
            total_size = int(response.headers.get('content-length', 0))
            with tqdm(total=total_size, unit='B', unit_scale=True, unit_divisor=1024) as bar:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        file.write(chunk)
                        bar.update(len(chunk))

        print(f"Image saved to {file_path}")

    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")

def extract_image_urls(webpage_url):
    try:
        response = requests.get(webpage_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        img_tags = soup.find_all('img')
        urls = [urljoin(webpage_url, img['src']) for img in img_tags if 'src' in img.attrs]
        return urls
    except requests.RequestException as e:
        print(f"Error fetching {webpage_url}: {e}")
        return []

def main():
    print("Image Downloader")

    # User input for URLs
    urls = input("Enter image URLs (comma-separated) or a webpage URL: ").split(',')

    # User input for save directory
    save_dir = input("Enter directory to save images: ").strip()

    # Process each URL
    for url in urls:
        url = url.strip()
        if url:
            # Check if URL is a webpage and extract images
            if urlparse(url).scheme in ['http', 'https'] and 'html' in url:
                image_urls = extract_image_urls(url)
                if image_urls:
                    for img_url in image_urls:
                        download_image(img_url, save_dir)
                else:
                    print("No images found on the provided webpage.")
            else:
                download_image(url, save_dir)

if __name__ == "__main__":
    main()


Image Downloader
